In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import teetool as tt
from teetool import visual_3d
import os

/home/will/anaconda3/envs/teetool/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
if not os.path.isfile('data/data_rocket_1.csv'):
    !wget "https://www.southampton.ac.uk/~wje1n13/teetool/example3.csv" -O "data/data_rocket_1.csv"

In [3]:
# SETTINGS

M = 200 # resampling (reduces points, doesn't interpolate), negative turns off
store_this_case_x = "d2 [-]" # choose dimension!

In [4]:
# read data

df = pd.read_csv("data/data_rocket_1.csv")

df.tail()

,INDEX,ID_STR,ID_STAGE_STR,TIME_SECONDS_DOUBLE,EASTINGS_METERS_DOUBLE,NORTHINGS_METERS_DOUBLE,ALTITUDE_METERS_DOUBLE,DISTANCE_METERS_DOUBLE,EVENT_INT
302294,654,99,LowerStage,61.9961,-236.962,1449.62,26.2735,1.450388e+06,2
302295,655,99,LowerStage,62.0549,-237.072,1450.25,14.6423,1.451858e+06,2
302296,656,99,LowerStage,62.1164,-237.188,1450.92,2.4534,1.453328e+06,2
302297,657,99,LowerStage,62.1590,-237.268,1451.37,-5.9788,1.454799e+06,2
302298,658,99,LowerStage,62.1690,-237.287,1451.48,-7.9569,1.456270e+06,3


In [5]:
# rename columns

df.rename(columns={'TIME_SECONDS_DOUBLE': 't [s]', 
                   'EASTINGS_METERS_DOUBLE': 'x [m]',
                   'NORTHINGS_METERS_DOUBLE': 'y [m]',
                   'ALTITUDE_METERS_DOUBLE': 'z [m]',
                   'DISTANCE_METERS_DOUBLE': 'd [m]'}, inplace=True)

In [6]:
# "is_event column

temp = np.diff(df["EVENT_INT"])

temp = np.concatenate(([0],temp))

df["is_event"] = (temp == 1)

In [7]:
# reduce to upper stage only
df_us = df[df["ID_STAGE_STR"] == "UpperStage"]

# reduce to first 30
df_us = df_us[df_us["ID_STR"] <= 30]

In [8]:
# normalise x y z GLOBALLY (all trajectories)

these_cells_input = ["x [m]", "y [m]", "z [m]"]
these_cells_output = ["x [-]", "y [-]", "z [-]"]

for (i, cell_input) in enumerate(these_cells_input):
    
    cell_output = these_cells_output[i]
    
    df_us.loc[:,cell_output] = (df_us[cell_input] - df_us[cell_input].min()) / ( df_us[cell_input].max() - df_us[cell_input].min() )


df_us.head()

,INDEX,ID_STR,ID_STAGE_STR,t [s],x [m],y [m],z [m],d [m],EVENT_INT,is_event,x [-],y [-],z [-]
0,0,0,UpperStage,0.0100,0.0,0.0,0.0,0.0,0,False,0.477557,0.0,0.000174
1,1,0,UpperStage,0.0120,0.0,0.0,0.0,0.0,0,False,0.477557,0.0,0.000174
2,2,0,UpperStage,0.0147,0.0,0.0,0.0,0.0,0,False,0.477557,0.0,0.000174
3,3,0,UpperStage,0.0182,0.0,0.0,0.0,0.0,0,False,0.477557,0.0,0.000174
4,4,0,UpperStage,0.0190,0.0,0.0,0.0,0.0,0,False,0.477557,0.0,0.000174


In [9]:
# normalise time / distance LOCALLY (per trajectory)

these_cells_input = ["t [s]", "d [m]"]
these_cells_output = ["t [-]", "d [-]"]

for (i, cell_input) in enumerate(these_cells_input):
    
    cell_output = these_cells_output[i]
    
    df_col = df_us[cell_input]
    
    temp = [];
    
    for i1 in df_us.ID_STR.unique():
        # this trajectory
        df1 = df_col[(df_us["ID_STR"] == i1)]
    
        temp1 = ( df1 - df1.min() ) / ( df1.max() - df1.min() )
        
        # concatenate
        temp = np.concatenate((temp, temp1.values), axis=0)
    
    # store altered dimension
    df_us[cell_output] = temp
    
# print result
df_us.head()

,INDEX,ID_STR,ID_STAGE_STR,t [s],x [m],y [m],z [m],d [m],EVENT_INT,is_event,x [-],y [-],z [-],t [-],d [-]
0,0,0,UpperStage,0.0100,0.0,0.0,0.0,0.0,0,False,0.477557,0.0,0.000174,0.000000,0.0
1,1,0,UpperStage,0.0120,0.0,0.0,0.0,0.0,0,False,0.477557,0.0,0.000174,0.000008,0.0
2,2,0,UpperStage,0.0147,0.0,0.0,0.0,0.0,0,False,0.477557,0.0,0.000174,0.000020,0.0
3,3,0,UpperStage,0.0182,0.0,0.0,0.0,0.0,0,False,0.477557,0.0,0.000174,0.000034,0.0
4,4,0,UpperStage,0.0190,0.0,0.0,0.0,0.0,0,False,0.477557,0.0,0.000174,0.000037,0.0


In [10]:
# find partitions

df_part = pd.DataFrame(columns = ["EVENT_INT","ID_STAGE_STR","DIM","MIN","MAX"])

these_dim = ["t [-]","d [-]"]

# use mean trajectory as guide
df1 = df_us[df_us["ID_STR"] == 0]

last_max_dic = {}

for this_dim in these_dim:
    last_max_dic[this_dim] = 0
    

for this_stage in df_us["ID_STAGE_STR"].unique():
    # this stage

    for this_event in df_us["EVENT_INT"].unique():
        # this event

        # consider only this segment of events
        df2 = df_us[df_us["EVENT_INT"] == this_event]

        for this_dim in these_dim:
            # this dimension

            # extract this dimension
            df3 = df2[this_dim]

            # extract minimum range of this segment
            event_min = last_max_dic[this_dim]

            # extract maximum range of this segment
            event_max = df3.max()
            
            # update last max
            last_max_dic[this_dim] = event_max

            # calculate different
            #event_diff = event_max - event_min

            # new addition
            df_temp = pd.DataFrame({"EVENT_INT":[this_event],
                                    "DIM":[this_dim],"MIN":[event_min],
                                    "MAX":[event_max],
                                   "ID_STAGE_STR":[this_stage]})

            # append addition
            df_part = df_part.append(df_temp,ignore_index=True)
            

print(df_part)



     DIM  EVENT_INT ID_STAGE_STR       MAX       MIN
0  t [-]        0.0   UpperStage  0.038107  0.000000
1  d [-]        0.0   UpperStage  0.007720  0.000000
2  t [-]        1.0   UpperStage  0.281762  0.038107
3  d [-]        1.0   UpperStage  0.415519  0.007720
4  t [-]        2.0   UpperStage  0.590153  0.281762
5  d [-]        2.0   UpperStage  0.972744  0.415519
6  t [-]        3.0   UpperStage  1.000000  0.590153
7  d [-]        3.0   UpperStage  1.000000  0.972744


In [11]:
# normalise time / distance SUPER LOCALLY (per trajectory, per segment)

these_cells_input = ["t [-]", "d [-]"]
these_cells_output = ["t2 [-]", "d2 [-]"]

for (i, this_dim) in enumerate(these_cells_input):
    
    # input / output combination
    cell_output = these_cells_output[i]
    
    # consider this dimension
    #this_dim = df_us[cell_input]
    
    for this_traj in df_us.ID_STR.unique():
        # this trajectory
        #df2 = df1[(df_us["ID_STR"] == this_traj)]
        
        for this_event in df_us.EVENT_INT.unique():
            # single segment/event
            #temp1 = df2[(df_us["EVENT_INT"] == this_segment)]
            
            these_rows = ((df_us["ID_STR"] == this_traj) &
                            (df_us["EVENT_INT"] == this_event))
            
            # extract data
            temp_data = df_us.loc[these_rows, this_dim]
                        
            # normalised array (one trajectory, one segment)
            temp_norm = ( temp_data - temp_data.min() ) / ( temp_data.max() - temp_data.min() )
            
            # extract information
                       
            these_rows2 = ((df_part["DIM"] == this_dim) &
                               (df_part["EVENT_INT"] == this_event))
            
            event_min = df_part.loc[these_rows2, "MIN"].values
            
            event_max = df_part.loc[these_rows2, "MAX"].values
            
            #print((event_min,event_max))
            
            # modify to fit stretch
            temp_fit = (event_max - event_min) * temp_norm + event_min
            
            # store new data in cell_output
            df_us.loc[these_rows, cell_output] = temp_fit
    
# print result
df_us.head()

,INDEX,ID_STR,ID_STAGE_STR,t [s],x [m],y [m],z [m],d [m],EVENT_INT,is_event,x [-],y [-],z [-],t [-],d [-],t2 [-],d2 [-]
0,0,0,UpperStage,0.0100,0.0,0.0,0.0,0.0,0,False,0.477557,0.0,0.000174,0.000000,0.0,0.000000,0.0
1,1,0,UpperStage,0.0120,0.0,0.0,0.0,0.0,0,False,0.477557,0.0,0.000174,0.000008,0.0,0.000009,0.0
2,2,0,UpperStage,0.0147,0.0,0.0,0.0,0.0,0,False,0.477557,0.0,0.000174,0.000020,0.0,0.000021,0.0
3,3,0,UpperStage,0.0182,0.0,0.0,0.0,0.0,0,False,0.477557,0.0,0.000174,0.000034,0.0,0.000036,0.0
4,4,0,UpperStage,0.0190,0.0,0.0,0.0,0.0,0,False,0.477557,0.0,0.000174,0.000037,0.0,0.000039,0.0


In [14]:
# flick data into arrays

# number of trajectories
N = df_us.ID_STR.unique().size

cluster_data = []

for (n, id_str) in enumerate(df_us.ID_STR.unique()):
    
    df1 = df_us[df_us["ID_STR"] == id_str]
    
    Yn = np.array([df1["x [m]"].values, df1["y [m]"].values, df1["z [m]"].values]).transpose()
    yn = np.reshape(Yn, (-1,1), order='F')
    xn = np.array(df1[store_this_case_x].values)
    
    # add noise
    (nrows, ncols) = Yn.shape
    Yn += 1*np.random.randn(nrows, ncols)
    
    # remove initial values
    #Yn = Yn[50:,:]
    #xn = xn[50:]
    
    # add tuple to list
    cluster_data.append((xn, Yn))

In [ ]:
# settings
#settings = {"model_type":"EM", "ngaus":100, "basis_type":"bernstein", "nbasis":5}
settings = {"model_type":"resampling", "ngaus":500}

# build world
world_name = "Cambridge Rocketry Simulator"

# build world
new_world = tt.World(name=world_name, ndim=3)

# modify default resolution
new_world.setResolution(xstep=50, ystep=50, zstep=50)

new_world.addCluster(cluster_data, "rocket trajectories")

# overview
new_world.overview()

# model
new_world.buildModel(0, settings)
new_world.overview()  # overview

# log
new_world.buildLogProbality(0)
new_world.overview()  # overview

# visuals by mayavi
visual = visual_3d.Visual_3d(new_world)
# visualise trajectories
visual.plotTrajectories([0])
# visualise intersection
visual.plotLogProbability([0])
# visualise outline
visual.plotOutline()
# wait to close
visual.show()


*** overview [Cambridge Rocketry Simulator] ***
0 [rocket trajectories] [-] [-]
*** overview [Cambridge Rocketry Simulator] ***
0 [rocket trajectories] [*] [-]
..............